In [1]:
#loading require python packages
import pandas as pd
import pickle
import socket
import json
from tkinter import messagebox
import numpy as np
from IPython.display import display

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor
import pickle
import os
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px


Matplotlib is building the font cache; this may take a moment.


In [ ]:
#using this code we are connecting to cloud server
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
client.connect(('localhost', 2222))
features = []
features.append("download")
features.append("Sales.csv")
features = pickle.dumps(features)
client.send(features) #now sending request to cloud to download sales data
data = client.recv(100000)#now receive data from cloud
data = pickle.loads(data)
data = data[0]
fout = open("Sales.csv", 'wb')
fout.write(data)
fout.close()
dataset = pd.read_csv("Sales.csv") #now display received dataset
display(dataset)

In [ ]:
#processing dataset to replace NAN values with 0
dataset.fillna(0, inplace = True)
dataset['YOY_Sales'] = dataset['YOY_Sales'].str.replace('%','').astype(float)
dataset['YOY_Units'] = dataset['YOY_Units'].str.replace('%','').astype(float)
dataset.drop(['Rank'], axis = 1,inplace=True)
dataset.drop(['Content'], axis = 1,inplace=True)
#here we are converting various menu items into categorries
dataset['category'] = dataset['Segment_Category'].apply(lambda x:x.replace('Quick Service & Burger','Burger').replace(
                                                      'Italian/Pizza','Pizza').replace('Quick Service & Pizza','Pizza').replace(
'Bakery Cafe ','Cafe').replace('Coffee Cafe','Cafe')).replace('Quick Service & Chicken','Chicken').replace('Casual Dining & Pizza','Pizza').replace(
'Quick Service & Cafe','Cafe').replace('Fast Casual & Pizza','Pizza').replace('Fast Casual & Bakery Cafe','Cafe').replace('Quick Service & Frozen Desserts','Dessert').replace(
'Quick Service & Family Casual','Family').replace('Casual Dining & Asian','Asian').replace('Asian/Noodle','Asian').replace('Casual Dining & Seafood','Seafood').replace(
'Bakery Cafe','Cafe').replace('Frozen Desserts','Dessert').replace('Family Dining & Family Style','Family').replace('Fast Casual & Sandwich','Sandwich').replace(
'Family Casual','Family').replace('Fast Casual & Chicken','Chicken').replace('Fast Casual & Burger','Burger').replace('Casual Dining & Steak','Steak').replace(
'Casual Dining & Sports Bar','Sports Bar').replace('Quick Service & Mexican','Mexican').replace('Quick Service & Mexican','Mexican').replace(
'Quick Service & Seafood','Seafood').replace('Quick Service & Sandwich','Sandwich').replace('Seafodd','Seafood').replace('Fine Dining & Steak','Steak').replace(
'Quick Service & Bakery Cafe','Cafe').replace('Fast Casual & Asian/Noodle','Asia').replace('Quick Service & Snack','Snack').replace(
'Fast Casual & Mexican','Mexican').replace('Quick Service & Beverage','Drinks').replace('Asian','Asia').replace('Sports Bar','Sports Bar').replace(
'Casual Dining & Varied Menu','Varied Menu').replace('Steak','Meat').replace('BBQ','Meat').replace('Family Style','Family')
display(dataset)

In [ ]:
#here we are plotting graph of Pizza sales in different restaurants so we can know which restaurant is popular for pizza 
#selling and has more sale compare to other restaurants and here we plot graph for 10 top pizza sale restaurant 
pizza = dataset.query("category == 'Pizza'")[:10]
fig = px.bar(pizza,
             x='Restaurant',
             y='Sales',
             color='Sales',
             color_continuous_scale='Viridis',
             title='Pizza Sales Top Restaurants')
fig.show()

In [ ]:
#here we are plotting graph of Burger sales in different restaurants so we can know which restaurant is popular for Burger
#selling and has more sale compare to other restaurants and here we plot graph for 10 top Burger sale restaurant
burger = dataset.query("category == 'Burger'")[:10]
fig = px.bar(burger,
             x='Restaurant',
             y='Sales',
             color='Sales',
             color_continuous_scale='Viridis',
             title='Burger Sales Top Restaurants')
fig.show()

In [ ]:
#frequency of different restaurant found in dataset
print("Frequency Table for Different Categories")
display(dataset['category'].value_counts())
names, count = np.unique(dataset['category'], return_counts=True)
plt.plot(names, count)
plt.xticks(rotation=90)
plt.rcParams["figure.figsize"]= [12,6]
plt.title("Frequency of Different Categories found in Dataset")
plt.show()

In [ ]:
#plotting graph for top 10 restaurant seller in all categories
dataset.groupby('Restaurant').agg({'Sales': 'max'})['Sales'].nlargest(10).plot(kind='bar')
plt.title("Top 10 Restaurants with highest sale")
plt.rcParams["figure.figsize"]= [12,6]
plt.show()

In [ ]:
'''
all machine learning algorithms accept only numeric values but this dataset contains mixed of numeric and non-numeric data 
so by applying label encoding algorithm we are converting all non-numeric values to numeric values. Label encoder will
assign integer values to all unique non-numeric data
'''
columns = ['Restaurant', 'category', 'Headquarters', 'Segment_Category']
label_encoder = []
for column in columns:
    le = LabelEncoder()
    dataset[column] = pd.Series(le.fit_transform(dataset[column].astype(str)))
    label_encoder.append(le)
display(dataset)

In [ ]:
#extracting Y label data
Y = dataset.values[:,1:2]
dataset.drop(['Sales'], axis = 1,inplace=True)
#extracting training X data
sc = MinMaxScaler(feature_range = (0, 1))
X = dataset.values
X = sc.fit_transform(X)
Y = sc.fit_transform(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)
print()
print("Dataset Size : "+str(X.shape[0]))
print("Train & Test Dataset Split Details")
print("Training Size : "+str(X_train.shape[0]))
print("Testing Size : "+str(X_test.shape[0]))
print()

In [ ]:
#defining Decision Tree regression object
dt_regression = DecisionTreeRegressor()
#training Decision Tree regression with X and Y data
dt_regression.fit(X_train, y_train)
#performing prediction on test data
predict = dt_regression.predict(X_test)
#calculating MSE error
dt_mse = mean_squared_error(y_test,predict)
dt_mse = format(dt_mse, '.7f')
dt_rmse = mean_squared_error(y_test,predict)
coefficient_determination = r2_score(y_test,predict)
rsquare = 1 - ((1 - coefficient_determination) * (len(X_test) - 1) / (len(X_test) - 1))
predict = predict.reshape(predict.shape[0],1)
predict = sc.inverse_transform(predict)
predict = predict.ravel()
labels = sc.inverse_transform(y_test)
labels = labels.ravel()
for i in range(len(y_test)):
    print("Test Data Sales: "+str(labels[i])+" Decision Tree Regression Predicted Sales : "+str(predict[i]))
print("\nDecision Tree Regression Mean Square Error: "+str(dt_mse))
#plotting comparison graph between original values and predicted values
plt.plot(labels, color = 'red', label = 'Original Test Sales')
plt.plot(predict, color = 'green', label = 'Decision Tree Regression Predicted Sales')
plt.title('Decision Tree Regression Sales Forecasting Graph')
plt.xlabel('Test Data')
plt.ylabel('Forecasting Sales')
plt.legend()
plt.show()

In [ ]:
#defining random forest regression object
rf_regression = RandomForestRegressor()
#training random forest regression with X and Y data
rf_regression.fit(X_train, y_train.ravel())
#performing prediction on test data
predict = rf_regression.predict(X_test)
#calculating MSE error
rf_mse = mean_squared_error(y_test,predict)
rf_mse = format(rf_mse, '.7f')
rf_rmse = mean_squared_error(y_test,predict)
coefficient_determination = r2_score(y_test,predict)
rsquare = 1 - ((1 - coefficient_determination) * (len(X_test) - 1) / (len(X_test) - 1))
predict = predict.reshape(predict.shape[0],1)
predict = sc.inverse_transform(predict)
predict = predict.ravel()
labels = sc.inverse_transform(y_test)
labels = labels.ravel()
for i in range(len(y_test)):
    print("Test Data Sales: "+str(labels[i])+" Random Forest Regression Predicted Sales : "+str(predict[i]))
print("\nRandom Forest Regression Mean Square Error: "+str(rf_mse))
#plotting comparison graph between original values and predicted values
plt.plot(labels, color = 'red', label = 'Original CTC')
plt.plot(predict, color = 'green', label = 'Random Forest Regression Predicted Sales')
plt.title('Random Forest Regression Sales Forecasting Graph')
plt.xlabel('Test Data')
plt.ylabel('Forecasting Sales')
plt.legend()
plt.show()

In [ ]:
height = [float(dt_mse), float(rf_mse)]
bars = ('Decision Tree', 'Random Forest')
y_pos = np.arange(len(bars))
plt.bar(y_pos, height)
plt.xticks(y_pos, bars)
plt.title("MSE Comparison Graph")
plt.rcParams["figure.figsize"]= [12,4]
plt.show()